# Information Robust Dirichlet Networks


In this notebook, we implement the specialized training loss proposed in the paper _Information Robust Dirichlet Networks for Predictive Uncertainty Estimation_ by Tsiligkaridis (2019). The method models predictive uncertainty by having a neural network output Dirichlet concentration parameters 𝛼 instead of just a pointwise softmax.

The total loss is composed of three terms:

1. Calibration term: implemented in the function  lp_fn
2. Regularization term: implemented in the function  regularization_fn
3. Adversiarial Entropy penalty: implemented in the function  dirichlet_entropy

In the paper and in this notenbook, L_p loss is not directly computed but rather an upper bound for it, denoted by F_i (for sample i)  

The regularization term penalizes high alpha values for incorrect classes.  

The final term uses the alpha values the model assigns to adversarial inputs.
The model is rewarded for outputting a Dirichlet-distribution with high entropy on these inputs.

### 1. The Loss

In [2]:
from __future__ import annotations

import torch
from torch.special import digamma


def lp_fn(alpha: torch.Tensor, y: torch.Tensor, p: float = 2.0) -> torch.Tensor:
    """Compute the Lp calibration loss (upper bound Fi).

    Computes F_i using the expectation-based formulation:
        F_i = ( E[(1-p_c)^p] + Σ_{j≠c} E[p_j^p] )^(1/p)

    Args:
        alpha: Dirichlet concentration parameters, shape (B, K), must be > 0
        y: One-hot encoded labels, shape (B, K)
        p: Lp norm exponent (default: 2.0)

    Returns:
        Scalar loss summed over batch

    Raises:
        ValueError: If alpha contains non-positive values or shapes don't match
    """
    if not torch.all(alpha > 0):
        msg = f"All alpha values must be > 0, got min={alpha.min().item()}"
        raise ValueError(msg)

    if alpha.shape != y.shape:
        msg = f"alpha and y shape mismatch: {alpha.shape} vs {y.shape}"
        raise ValueError(msg)

    _B, _K = alpha.shape  # noqa: N806

    # total concentration alpha0
    alpha0 = alpha.sum(dim=1, keepdim=True)  # (B,1)

    # extract alpha_c (correct class)
    alpha_c = (alpha * y).sum(dim=1, keepdim=True)  # (B,1)
    alpha0_minus_c = alpha0 - alpha_c  # (B,1)

    # log B(a,b) used for expectations: E[X^p] = B(a+p,b)/B(a,b)
    def log_b(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        return torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a + b)

    # E[(1 - p_c)^p]   where (1 - p_c) ~ Beta( alpha0 - alpha_c , alpha_c )
    log_e1 = log_b(alpha0_minus_c + p, alpha_c) - log_b(alpha0_minus_c, alpha_c)
    e1 = torch.exp(log_e1)  # (B,1)

    # Per-class E[p_j^p] for all j
    log_ep = log_b(alpha + p, alpha0 - alpha) - log_b(alpha, alpha0 - alpha)
    ep = torch.exp(log_ep)

    # zero-out the true class term so we sum only j≠c
    ep = ep * (1 - y)

    # final expectation sum
    e_sum = e1 + ep.sum(dim=1, keepdim=True)  # (B,1)

    # apply ^(1/p)  # noqa: ERA001
    fi = torch.exp(torch.log(e_sum + 1e-8) / p).squeeze(1)  # (B,)

    return fi.sum()


def regularization_fn(alpha: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Compute the regularization term using trigamma functions.

    Penalizes high alpha values for incorrect classes to encourage confident
    but calibrated predictions.

    Args:
        alpha: Dirichlet concentration parameters, shape (B, K), must be > 0
        y: One-hot encoded labels, shape (B, K)

    Returns:
        Scalar regularization loss

    Raises:
        ValueError: If shapes don't match
    """
    if alpha.shape != y.shape:
        msg = f"alpha and y shape mismatch: {alpha.shape} vs {y.shape}"
        raise ValueError(msg)

    # Build alpha_tilde by replacing correct-class alpha with 1
    alpha_tilde = alpha * (1 - y) + y

    # Compute alpha_tilde_0 = 1 + sum over incorrect classes
    alpha_tilde_0 = torch.sum(alpha_tilde, dim=1, keepdim=True)

    # Polygamma(1, x) = trigamma(x)
    trigamma_alpha = torch.polygamma(1, alpha_tilde)
    trigamma_alpha0 = torch.polygamma(1, alpha_tilde_0)

    # (alpha_tilde - 1)^2 term
    diff_sq = (alpha_tilde - 1.0) ** 2

    # Penalty only for incorrect classes → mask out true class
    mask = 1 - y

    # Compute elementwise contribution
    term = 0.5 * diff_sq * (trigamma_alpha - trigamma_alpha0) * mask

    # Sum over classes and batch
    return torch.sum(term)


def dirichlet_entropy(alpha: torch.Tensor) -> torch.Tensor:
    """Compute Dirichlet entropy.

    For adversarial examples, we want to maximize entropy (reward the model for
    being uncertain), which appears as a negative term in the loss.

    Entropy formula (a stands for alpha):
        H(a) = log B(a) + (a_0 - K) * ψ(a_0) - Σ_k (a_k - 1) * ψ(a_k)

    Args:
        alpha: Dirichlet concentration parameters, shape (B_a, K), must be > 0

    Returns:
        Scalar entropy summed over batch

    Raises:
        ValueError: If alpha contains non-positive values
    """
    if not torch.all(alpha > 0):
        msg = f"All alpha values must be > 0, got min={alpha.min().item()}"
        raise ValueError(msg)

    K = alpha.size(-1)  # noqa: N806
    alpha0 = alpha.sum(dim=-1)

    log_b = torch.lgamma(alpha).sum(dim=-1) - torch.lgamma(alpha0)

    term1 = log_b
    term2 = (alpha0 - K) * digamma(alpha0)
    term3 = ((alpha - 1) * digamma(alpha)).sum(dim=-1)
    entropy = term1 + term2 - term3

    return entropy.sum()


def loss_IRD(  # noqa: N802
    alpha: torch.Tensor,
    y: torch.Tensor,
    adversarial_alpha: torch.Tensor | None = None,
    p: float = 2.0,
    lam: float = 1.0,
    gamma: float = 1.0,
    normalize: bool = True,
) -> torch.Tensor:
    """Compute the Loss introduced in paper: Information Robust Dirichlet Networks for Predictive Uncertainty Estimation
    Args:
        alpha : (B, K) Dirichlet concentration parameters
        adversarial_alpha : (B_a, K) adversarial_alpha concentration parameters for adversarial inputs
        y     : (B, K) one-hot labels
        p     : scalar exponent
    Returns:
        loss_IRD : the IRD loss comprised of all three terms, summed over all input examples.
    """  # noqa: D205
    # Input validation
    if alpha.dim() != 2 or y.dim() != 2:
        msg = f"alpha and y must be 2D, got {alpha.dim()}, {y.dim()}"
        raise ValueError(msg)

    if alpha.shape != y.shape:
        msg = f"alpha and y shape mismatch: {alpha.shape} vs {y.shape}"
        raise ValueError(msg)

    if not torch.all(alpha > 0):
        msg = f"All alpha values must be > 0, got min={alpha.min().item()}"
        raise ValueError(msg)

    # Compute Loss Components
    lp_term = lp_fn(alpha, y, p)
    reg_term = regularization_fn(alpha, y)

    if adversarial_alpha is not None:
        if adversarial_alpha.dim() != 2:
            msg = f"adversarial_alpha must be 2D, got {adversarial_alpha.dim()}"
            raise ValueError(msg)

        if adversarial_alpha.shape[1] != alpha.shape[1]:
            msg = (
                f"adversarial_alpha must have same number of classes as alpha: "
                f"{adversarial_alpha.shape[1]} vs {alpha.shape[1]}"
            )
            raise ValueError(
                msg,
            )

        entropy_term = dirichlet_entropy(adversarial_alpha)
    else:
        entropy_term = 0.0

    # Normalize by batch sizes for stable training across different batch sizes
    if normalize:
        B = alpha.shape[0]  # noqa: N806
        K = alpha.shape[1]  # noqa: N806
        lp_term = lp_term / B
        reg_term = reg_term / (B * K)

        if adversarial_alpha is not None and isinstance(entropy_term, torch.Tensor):
            B_a = adversarial_alpha.shape[0]  # noqa: N806
            entropy_term = entropy_term / B_a

    loss = lp_term + lam * reg_term - gamma * entropy_term

    return loss

### 2. Testing

Let´s also test our loss by implementing a simple Evidential CNN for images. Here is the setup:

#### Let´s assume the user passes a dataset to us
In this example we use CIFAR10 as an example, but the code works for any torch.util.data.Dataset class.

If the user has a dataset, which he passes to us, we can automatically adapt the models input channels and output dimension (num_classes), based on the dataset.

In [3]:
# example dataset
from torch import nn
import torch.nn.functional as F  # noqa: F401
from torch.utils.data import DataLoader  # noqa: F401
from torchvision import datasets, transforms
import tqdm

transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ],
)

dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

# wrap into a DataLoader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

#### Determine the input channels and number of class labels

In [20]:
def get_num_classes(dataset: torch.utils.data.Dataset) -> int:
    """Determine number of classes by checking in order:
    1) dataset.classes
    2) dataset.targets
    3) full dataset scan (max label + 1).
    """  # noqa: D205
    # Try Torchvision-style classes attribute
    if hasattr(dataset, "classes"):
        try:
            return len(dataset.classes)
        except Exception:  # noqa: BLE001, S110
            pass

    # Else try Torchvision-style targets tensor
    if hasattr(dataset, "targets"):
        targets = dataset.targets
        if isinstance(targets, torch.Tensor):
            return int(targets.max().item()) + 1
        else:  # noqa: RET505
            return max(targets) + 1

    # Fallback: scan entire dataset
    max_label = -1
    for _, y in dataset:
        if isinstance(y, torch.Tensor):
            y = y.item()  # noqa: PLW2901
        max_label = max(max_label, int(y))

    return max_label + 1


# to get the number of input channels, take the first image from the dataset and check its shape
c_in, H, W = dataset[0][0].shape  # assuming dataset returns (image, label) tuples

# to get the number of classes, use the get_num_classes function
num_classes = get_num_classes(dataset)

print(f"Input shape: ({c_in}, {H}, {W}), Number of classes: {num_classes}")

Input shape: (3, 32, 32), Number of classes: 10


#### Initialize a CNN model with correct input channels, feedforward dimension and output dimension

The CNN is composed of three layers, whereas the outputs of these layers are being transformed into alpha values (applying ReLU and then adding 1 instead of softmax) for evidential learning.

In [21]:
class CNN(nn.Module):
    """Simple Evidential CNN for images.
    Returns Dirichlet parameters (alpha) as output.
    """  # noqa: D205

    def __init__(self, c_in: int, num_classes: int) -> None:
        """Initialize the CNN."""
        super().__init__()
        # (B, C_in, H, W) -> (B, 32, H_out, W_out)
        self.conv1 = nn.Conv2d(in_channels=c_in, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU(inplace=True)

        # Dummy forward pass through the convolutions to infer feature dimension of ffwd layer automatically
        with torch.no_grad():
            dummy = torch.zeros(1, c_in, H, W)
            dummy = self._forward_conv(dummy)
            feature_dim = dummy.view(1, -1).size(1)

        self.fc1 = nn.Linear(feature_dim, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def _forward_conv(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass."""
        # Convolutions
        x = self._forward_conv(x)

        # Flatten and fully connected
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        # Turn outputs into alpha values for evidential learning
        x = self.relu(x)
        x = x + torch.ones_like(x)

        return x

#### Simple forward pass test to verify model dimensions are correct

In [22]:
# simple forward pass test
model = CNN(c_in=c_in, num_classes=num_classes)
sample_input = torch.randn(8, c_in, 32, 32)  # batch of 8 images
sample_output = model(sample_input)
print("Sample output shape:", sample_output.shape)  # should be (8, num_classes)

Sample output shape: torch.Size([8, 10])


We train the model using different parameters like the model we are using, the loss function we are using (criterion), or the number of training epochs, etc. 

We also add different uncertainty metrics, the predictive entropy (aleatoric & epistemic uncertainty) and mutual information (epistemic uncertainty). Using these metrics, we can further estimate the correctness and uncertainty of our model. The value of the entropy for the uniform distribution over 10 classes is around 2.3. This is the highest uncertainty the model can assign, so the values of the predictive entropy should be around 2.3, never higher.

In [ ]:
def train(model, optimizer, criterion, train_loader, val_loader, device, n_epochs=5) -> None:  # noqa: ANN001, D417
    """Train a model, validate after each epoch.

    Args:
        model: any evidential model
        criterion: this is your loss function, has to take inputs alpha and y as shape (B,)
          (float, optional):  bda parameter for the regularization term. Defaults to 0.15.
        n_epochs (int, optional): Number of training epochs. Defaults to 5.
    """
    model.train()
    for epoch in range(n_epochs):
        pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{n_epochs}")
        for x, y in pbar:
            x = x.to(device)  # noqa: PLW2901
            # Convert labels to one-hot encoding
            y = nn.functional.one_hot(y, num_classes=10).float()  # noqa: PLW2901
            y = y.to(device)  # noqa: PLW2901

            alpha = model(x)
            loss = criterion(alpha, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix({"loss": f"{loss.item():.4f}"})
        validate(model, val_loader, criterion, device)


def validate(model, val_loader, criterion, device) -> None:  # noqa: ANN001, ARG001
    """Validation loop."""
    model.eval()
    torch.manual_seed(42)

    all_pe_id = []
    all_mi_id = []
    all_pe_ood = []
    all_mi_ood = []

    with torch.no_grad():
        for x, y in val_loader:  # noqa: B007
            x = x.to(device)  # noqa: PLW2901

            # In-distribution
            alpha_id = model(x)
            pe_id, mi_id = dirichlet_mi(alpha_id)
            all_pe_id.append(pe_id)
            all_mi_id.append(mi_id)

            # OOD: Permuted MNIST
            B = x.shape[0]  # noqa: N806
            # random permutation of pixels
            perm = torch.randperm(28 * 28, device=device)
            x_permuted = x.reshape(B, 1, -1)[:, :, perm]
            x_permuted = x_permuted.reshape(B, 1, 28, 28)

            alpha_ood = model(x_permuted)
            pe_ood, mi_ood = dirichlet_mi(alpha_ood)
            all_pe_ood.append(pe_ood)
            all_mi_ood.append(mi_ood)

        # Concatenate tensors
        pe_id = torch.cat(all_pe_id)
        mi_id = torch.cat(all_mi_id)
        pe_ood = torch.cat(all_pe_ood)
        mi_ood = torch.cat(all_mi_ood)

        print("=== Uncertainty Summary ===")
        print(f"ID  — Predictive Entropy: mean {pe_id.mean().item():.4f}, std {pe_id.std().item():.4f}")
        print(f"ID  — Mutual Information: mean {mi_id.mean().item():.4f}, std {mi_id.std().item():.4f}")
        print(f"OOD — Predictive Entropy: mean {pe_ood.mean().item():.4f}, std {pe_ood.std().item():.4f}")
        print(f"OOD — Mutual Information: mean {mi_ood.mean().item():.4f}, std {mi_ood.std().item():.4f}")
        return {
            "pe_id": pe_id,
            "mi_id": mi_id,
            "pe_ood": pe_ood,
            "mi_ood": mi_ood,
        }


def dirichlet_mi(alpha: torch.Tensor):  # noqa: ANN201
    """Computes predictive entropy and mutual information for a Dirichlet prior.

    Args:
        alpha: (B, K) Dirichlet concentration

    Returns:
        predictive_entropy: (B,)
        mutual_information: (B,)
    """
    eps = 1e-12
    alpha = torch.clamp(alpha, min=1e-6)
    alpha0 = alpha.sum(dim=1, keepdim=True)  # (B,1)

    # Predictive probabilities
    p = alpha / alpha0

    # Predictive entropy H[Y]
    predictive_entropy = -(p * torch.log(p + eps)).sum(dim=1)

    # Expected conditional entropy E_p[H[Y|p]]
    digamma_alpha = torch.digamma(alpha + 1.0)
    digamma_alpha0 = torch.digamma(alpha0 + 1.0)  # (B,1)

    expected_cond_entropy = -torch.sum(
        (alpha / alpha0) * (digamma_alpha - digamma_alpha0),
        dim=1,
    )

    # Mutual information = H[pred] - E[cond]
    mutual_information = predictive_entropy - expected_cond_entropy

    return predictive_entropy, mutual_information

Now, we implement the function responsible for calculating the loss, accuracy and confidence for in-distribution samples and the confidence for out-of-distribution samples. The confidence for the OOD-samples should be lower than the confidence for the ID-samples.

In [ ]:
# Test for OOD inputs
@torch.no_grad()
def evaluate(model, criterion, data_loader, device) -> None:  # noqa: ANN001
    """Evaluate model on given data_loader."""
    model.eval()
    total_loss_id = 0.0
    correct_id = 0
    confidence_id = 0.0
    confidence_ood = 0.0
    length = len(data_loader.dataset)  # Number of samples

    for x, y in data_loader:
        x = x.to(device)  # noqa: PLW2901
        y = nn.functional.one_hot(y, num_classes=10).float()  # noqa: PLW2901
        y = y.to(device)  # noqa: PLW2901

        # OOD Noise inputs
        noise = torch.randn_like(x).to(device)
        alpha_noise = model(noise)

        # Calculate loss for in-distribution
        alpha = model(x)  # (B, num_classes)
        total_loss_id += criterion(alpha, y)

        # Calculate accuracy for in-distribution inputs
        pred = torch.max(alpha, -1).indices
        y_labels = torch.argmax(y, -1)
        correct_id += (pred == y_labels).sum().item()

        # Calculate confidence for in-distribution and OOD inputs
        confidence_id += (torch.max(alpha, -1).values / torch.sum(alpha, -1)).sum().item()
        confidence_ood += (torch.max(alpha_noise, -1).values / torch.sum(alpha_noise, -1)).sum().item()

    total_loss_id /= length
    accuracy_id = correct_id / length
    confidence_ood /= length
    confidence_id /= length

    print("Loss In-Distribution: ", total_loss_id.item())
    print("Confidence In-Distribution: ", confidence_id)
    print("Confidence OOD: ", confidence_ood)  # Ideally should be low
    print(f"Evaluation In-Distribution Accuracy: {accuracy_id:.4f}")

In the following code, we will be plotting a histogram which should visualize our uncertainty for ID and OOD-data.

In [ ]:
import matplotlib.pyplot as plt


def plot_uncertainty(pe_id, pe_ood, mi_id, mi_ood) -> None:  # noqa: ANN001
    plt.figure(figsize=(12, 5))

    # Predictive Entropy
    plt.subplot(1, 2, 1)
    plt.hist(
        pe_id.cpu().numpy(),
        bins=50,
        density=True,
        alpha=0.6,
        label="ID",
        color="#4C72B0",
    )
    plt.hist(
        pe_ood.cpu().numpy(),
        bins=50,
        density=True,
        alpha=0.6,
        label="OOD",
        color="#DC1489",
    )
    plt.xlabel("Predictive Entropy")
    plt.ylabel("Density")
    plt.title("Predictive Entropy: ID vs OOD")
    plt.legend()

    # Mutual Information
    plt.subplot(1, 2, 2)
    plt.hist(
        mi_id.cpu().numpy(),
        bins=50,
        density=True,
        alpha=0.6,
        label="ID",
        color="#4C72B0",
    )
    plt.hist(
        mi_ood.cpu().numpy(),
        bins=50,
        density=True,
        alpha=0.6,
        label="OOD",
        color="#DC1489",
    )
    plt.xlabel("Mutual Information")
    plt.ylabel("Density")
    plt.title("Mutual Information: ID vs OOD")
    plt.legend()

    plt.tight_layout()
    plt.show()

Now we implement the main function. We set the parameters and define the criterion we are using. In my case, that is the loss proposed in the paper _Information Robust Dirichlet Networks_ by Tsiligkaridis (2019).

In [ ]:
def main() -> None:
    """This code presumes that the loss function takes in alpha and y with shape (B, 10).
    Currently loss_IRD takes only one-hot encoded y
    criterion = loss_IRD.
    """  # noqa: D205
    # --------------- Standard setup --------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nDevice: {device}")
    print("\nLoading MNIST dataset...")

    # Use large batch to reduce gradient noise
    # Gradients will be noisy if regularization term is used

    # TODO(<julia>): Replace with own dataloaders and val_loaders and then uncomment the associated lines  # noqa: TD003
    # train_loader, val_loader = get_mnist_dataloaders(batch_size=128)  # noqa: ERA001

    print("\nInitialize model...")
    model = CNN(num_classes=10)
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)  # noqa: F841

    # Import your loss function with hyperparameters
    from src.probly.losses.evidential.torch import ird_loss  # noqa: PLC0415

    criterion = ird_loss(p=2.0, lam=0.01, gamma=0.01)  # noqa: F841

    # Train for a few epochs
    print("\nStarting training...")

    # train(model, optimizer, criterion, train_loader, val_loader, device, n_epochs=10)  # noqa: ERA001
    # evaluate(model, criterion, val_loader, device)  # noqa: ERA001

    # stats = validate(model, val_loader, criterion, device)  # noqa: ERA001
    # plot_uncertainty(
    # stats["pe_id"],  # noqa: ERA001
    # stats["pe_ood"],  # noqa: ERA001
    # stats["mi_id"],  # noqa: ERA001
    # stats["mi_ood"],  # noqa: ERA001
    # )  # noqa: ERA001, RUF100

In [ ]:
main()

To enhance code reusability and maintain a clean notebook structure, we can get access to the Grayscale-CNN by importing it from the dedicated module.

The same goes for the IRD loss. As seen below, we implement it from train.evidential.torch, which contains all the different losses introduced in the papers.

- **Model Architecture:** Import `GrayscaleMNISTCNN` from `probly.layers.evidential.torch`
- **Loss Function:** Import `ird_loss` from `probly.losses.evidential.torch`  
- **Training Interface:** Use `unified_evidential_train` for standardized training

In [ ]:
# ============================================================================
# Advanced: Unified Evidential Training Interface (Optional)
# ============================================================================
# NOTE: Use main() above instead. This is for advanced testing only.
# Uncomment to test the unified training interface:


from probly.losses.evidential.torch import ird_loss
from probly.models.evidential.torch import IRDModel
from probly.train.evidential.torch import unified_evidential_train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class DirichletMLPEncoder(nn.Module):
    """Simple MLP encoder for transforming inputs into feature embeddings.

    This module contains no evidential logic, only feature extraction.
    """

    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 128,
        latent_dim: int = 128,
    ) -> None:
        """Initialize the MLP encoder.

        Args:
            input_dim: Size of input features (flattened or 1D).
            hidden_dim: Number of neurons in hidden layers (default: 128).
            latent_dim: Dimension of output feature representation (default: 128).
        """
        super().__init__()
        self.latent_dim = latent_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Compute feature embedding.

        Args:
            x: Input tensor of shape (batch_size, input_dim).

        Returns:
            Feature tensor of shape (batch_size, latent_dim).
        """
        return self.net(x)


# train_loader, val_loader = get_mnist_dataloaders(batch_size=128)  # noqa: ERA001
enc = DirichletMLPEncoder(input_dim=28 * 28)
model = IRDModel(encoder=enc)
criterion = ird_loss

# Call unified_evidential_train with correct parameters
unified_evidential_train(
    mode="IRD",
    model=model,
    dataloader=dataloader,
    loss_fn=criterion,
    epochs=10,
    lr=1e-3,
    device=device,
)